In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib.dates as mdates
import os

## 1 weekdays

In [2]:
os.chdir('C:\\Users\\alexx\\Documents\\thesis\\processed data')
dfALL = pd.read_csv('SALES.csv')
dfALL.reset_index(level=0, inplace=True)
dfVR = pd.DataFrame(dfALL['date'])

In [3]:
df_day_of_week = pd.DataFrame(dfVR['date'].copy())

In [4]:
df_day_of_week['date'] = pd.to_datetime(df_day_of_week['date'])
df_day_of_week['day_of_week'] = df_day_of_week['date'].dt.day_name()

def weekend(row):
    if row['day_of_week'] == 'Saturday' or row['day_of_week'] == 'Sunday':
        val = 1
    else:
        val = 0
    return val
df_day_of_week['weekend'] = df_day_of_week.apply(weekend, axis=1)

In [5]:
df_day_of_week['date'] = df_day_of_week['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_day_of_week = df_day_of_week.set_index('date')

In [6]:
df_day_of_week = pd.get_dummies(df_day_of_week)

In [7]:
dfVR = dfVR.set_index('date')
dfVR = pd.concat([dfVR, df_day_of_week], axis=1)

## 2 Payday
Usually, Japanese emplolyees get paid on the either 5th, 10th, 15th, 20th, 25th, or 30th of each month. So here we create variable "payday" to represent these 6 days of each month as 1; other days are represented as 0.

In [8]:
def payday_5(row):
    if row['date'][-1]=='5' and row['date'][-2] =='0' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

def payday_10(row):
    if row['date'][-1]=='1' and row['date'][-2]=='0' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

def payday_15(row):
    if row['date'][-1]=='1' and row['date'][-2]=='5' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

def payday_20(row):
    if row['date'][-1]=='2' and row['date'][-2]=='0' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

def payday_25(row):
    if row['date'][-1]=='2' and row['date'][-2]=='5' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

def payday_30(row):
    if row['date'][-1]=='3' and row['date'][-2]=='0' and row['day_of_week_Saturday']==0 and row ['day_of_week_Saturday']==0:
        val = 1
    else:
        val = 0
    return val

In [9]:
dfVR.reset_index(level=0, inplace=True)
dfVR['payday_5'] = dfVR.apply(payday_5, axis=1)
dfVR['payday_10'] = dfVR.apply(payday_10, axis=1)
dfVR['payday_15'] = dfVR.apply(payday_15, axis=1)
dfVR['payday_20'] = dfVR.apply(payday_20, axis=1)
dfVR['payday_25'] = dfVR.apply(payday_25, axis=1)
dfVR['payday_30'] = dfVR.apply(payday_30, axis=1)

In [10]:
# now add paydays that are delayed because it is on weekends
dfVR.loc[dfVR.date == '2019-12-16', 'payday_15'] = 1
dfVR.loc[dfVR.date == '2020-01-06', 'payday_5'] = 1
dfVR.loc[dfVR.date == '2020-01-27', 'payday_25'] = 1
dfVR.loc[dfVR.date == '2020-02-17', 'payday_15'] = 1
dfVR.loc[dfVR.date == '2020-03-16', 'payday_15'] = 1
dfVR.loc[dfVR.date == '2020-04-06', 'payday_5'] = 1
dfVR.loc[dfVR.date == '2020-04-27', 'payday_25'] = 1
dfVR.loc[dfVR.date == '2020-05-11', 'payday_10'] = 1

# lag1 for paydays
dfVR['payday_5_lag1'] = dfVR['payday_5'].shift(periods=1)
dfVR['payday_10_lag1'] = dfVR['payday_10'].shift(periods=1)
dfVR['payday_15_lag1'] = dfVR['payday_15'].shift(periods=1)
dfVR['payday_20_lag1'] = dfVR['payday_20'].shift(periods=1)
dfVR['payday_25_lag1'] = dfVR['payday_25'].shift(periods=1)
dfVR['payday_30_lag1'] = dfVR['payday_30'].shift(periods=1)

# combine
dfVR['payday_5'] = dfVR['payday_5'] + dfVR['payday_5_lag1']
dfVR['payday_10'] = dfVR['payday_10'] + dfVR['payday_10_lag1']
dfVR['payday_15'] = dfVR['payday_15'] + dfVR['payday_15_lag1']
dfVR['payday_20'] = dfVR['payday_20'] + dfVR['payday_20_lag1']
dfVR['payday_25'] = dfVR['payday_25'] + dfVR['payday_25_lag1']
dfVR['payday_30'] = dfVR['payday_30'] + dfVR['payday_30_lag1']

## 3 Weather & Air Quality

In [11]:
df_t = pd.read_excel('C:\\Users\\alexx\\Documents\\thesis\\raw data\\weather.xlsx').iloc[:,:]
df_t['PM2.5'].interpolate(method='linear', direction='forward', inplace=True)
df_t['date'] = df_t['date'].apply(lambda x: x.strftime('%Y-%m-%d'))

In [12]:
df_t = df_t.iloc[3:,:]
df_t = df_t.set_index('date')
dfVR = dfVR.set_index('date')
dfVR = pd.concat([dfVR, df_t], axis=1)

## 4 Holiday

In [13]:
holi = ['2019-12-28','2019-12-29','2019-12-30','2019-12-31','2020-01-01',
        '2020-01-02','2020-01-03','2020-01-04','2020-01-05','2020-01-13']

def holiday(row):
    if row['date'] in holi:
        val = 1
    else:
        val = 0
    return val
dfVR.reset_index(level=0, inplace=True)
dfVR['holiday'] = dfVR.apply(holiday, axis=1)

In [14]:
dfVR = dfVR.set_index('date')

## 5 Events 

In [16]:
df_event = pd.read_excel('C:\\Users\\alexx\\Documents\\thesis\\raw data\\events.xlsx')
df_event['date'] = df_event['date'].apply(lambda x: x.strftime('%Y-%m-%d'))
df_event = df_event.set_index('date')

## 6 Output All Variables

In [17]:
dfVR = pd.concat([dfVR, df_event], axis=1)
dfVR = dfVR.replace([np.inf, -np.inf], np.nan)
dfVR = dfVR.fillna(0)

In [18]:
dfVR.to_csv('Other_variables.csv')